In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


# First lets look at the GOS

In [2]:
# %%timeit -r 1
nviews = 100 #493 

s = fc.Spectrum()
s.load('Varian_truebeam')
s.x[0] = 1; s.x[1] = 2
s.attenuate(0.,fc.get_mu(z=13)) #3.7

kernel = fc.Kernel(s,'CuGOS-784-micrometer')
kernel.add_focal_spot(1.2)
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,mgy=1.1,bowtie=True,filter='FF0')
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[5].shape,r=5.7,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[5],im)

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 1210.1730333562502 number of photons 0.02636808212587352


## Now lets look at the bcca data

In [30]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_6x_300_rared.npy')
recon_slice = np.rot90(np.mean(bcca_recon.T[35:43],0),2)
im = create_mask(recon_slice.shape,off =[-0.15,0], r= 5.75,radius=0.3)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

res = minimize(HU_fun,[contrast_fc[0],contrast_fc[5]+0.003],args=(contrast_fc,contrast_bcca))

x = [.853,0.945,.998,1.147,1.36,1.87]
contrast_fc_scaled = (contrast_fc - res.x[0])*(res.x[1])
image_fc_scaled = (phantom.img[4]- res.x[0])*(res.x[1])

plt.figure(figsize=[8,3])
plt.subplot(131)
plt.imshow(recon_slice,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(132)
plt.imshow(image_fc_scaled,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(133)
plt.plot(x,contrast_fc_scaled,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,contrast_bcca,'s',c='darkorange',label='Star-lack')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.imsave('im_for_imagej.png',np.hstack((recon_slice,image_fc_scaled)))
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


# Now lets look at the kV data 

In [54]:
# %%timeit -r 1
nviews = 100 #887 # That is how many are in the file 
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(0.6,fc.get_mu(z=13))
s.attenuate(0.089,fc.get_mu(z=22))
kernel = fc.Kernel(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,scat_on=True,mgy=0.001,bowtie=True,filter='bowtie_mod')
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[4].shape,r=5.75,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[4],im)

(19, 19, 19) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 18.63455421000987 number of photons 0.6662745014665195


In [44]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kv_jan_2021.npy')
recon_slice = np.rot90(bcca_recon.T[50]).copy()
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

res = minimize(HU_fun,[contrast_fc[0],contrast_fc[3]],args=(contrast_fc,contrast_bcca),method='Powell')
fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (phantom.img[4] - res.x[0])*(res.x[1])

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
plt.plot(np.rot90(np.mean(bcca_recon.T[25:36],0),3)[256,:])
plt.plot(np.rot90(fc_image,0)[256,:])
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)

In [33]:
plt.imsave('data/im_for_imagej.png',np.hstack((recon_slice,fc_image)))
!imagej data/im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'
